<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/usage_example(colab)V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/skywalker0803r/c620.git
!pip install autorch > log.txt
!pip install optuna > log.txt

Cloning into 'c620'...
remote: Enumerating objects: 2154, done.
remote: Counting objects: 100% (972/972), done.
remote: Compressing objects: 100% (514/514), done.
remote: Total 2154 (delta 631), reused 686 (delta 449), pack-reused 1182
Receiving objects: 100% (2154/2154), 51.68 MiB | 26.62 MiB/s, done.
Resolving deltas: 100% (1409/1409), done.


# import

In [2]:
from c620.FV2 import AllSystem
from c620.configV2 import config
import joblib
import numpy as np
import pandas as pd
import os
os.chdir('/content/c620')
print(os.getcwd())
!git pull

/content/c620
Already up to date.


# demo data

In [3]:
demo = joblib.load('./data/demo.pkl')
print(demo.keys())
icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']
icg_input.head()

dict_keys(['icg_input', 'c620_feed', 't651_feed', 'c620_op', 'c620_wt', 'c660_op', 'c660_wt', 'c670_op', 'c670_wt'])


,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
140-023,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,41.0,0.0911,20.854397,74.689774,70,980,1.75,10,36.0


In [4]:
icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
140-023,980


In [5]:
icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
140-023,70


In [6]:
icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw
140-023,10


# 1.試算模式

In [7]:
f = joblib.load('/content/c620/model/allsystem.pkl')

In [8]:
demo['icg_input']

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
140-023,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,41.0,0.0911,20.854397,74.689774,70,980,1.75,10,36.0


In [9]:
c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = f.inference(demo['icg_input'],demo['c620_feed'],demo['t651_feed'])

(1, 3)
(1, 41)
(1, 10)
(1, 2)
(1, 41)
(1, 10)


# 檢查三項約束條件

In [10]:
c620_wt[['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]

,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
140-023,69.163988


In [11]:
na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000

981.4707079837214

In [12]:
c660_wt['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000

9.062412707267518

In [27]:
c620_wt.iloc[:,:41].sum(axis=1)

140-023    100.0
dtype: float64

In [28]:
c660_wt.iloc[:,:41].sum(axis=1)

140-023    100.0
dtype: float64

In [29]:
c670_wt.iloc[:,:41].sum(axis=1)

140-023    100.0
dtype: float64

# 推薦模式

In [13]:
c620_wt2,c620_op2,c660_wt2,c660_op2,c670_wt2,c670_op2,bz_error,nainbz_error,tol_error = f.recommend(demo['icg_input'],demo['c620_feed'],demo['t651_feed'])

[I 2021-04-23 08:35:48,152] A new study created in memory with name: no-name-93ef96df-0572-4961-8bc8-518d1218f6b8


(1, 3)
(1, 41)
(1, 10)
(1, 2)
(1, 41)
(1, 10)


[I 2021-04-23 08:35:48,202] Trial 0 finished with value: 7.920756232644692 and parameters: {'Density_Feed Properties': 0.8631149991214732, 'Density_Vent Gas Production Rate and Composition': 0.43190809857650736, 'Density_Distillate Production Rate and Composition': 0.8228154387618835, 'Density_Sidedraw Production Rate and Composition': 0.8791648399620315, 'Density_Bottoms Production Rate and Composition': 0.8720473536400659, 'Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr': 102.10296849082644, 'Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr': 12.78759559267168, 'Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr': 15.288980617123435, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 177.27987962642925, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 194.0799652601281}. Best is trial 0 with value: 7.920756232644692.
[I 2021-04-23 08:35:48,269] Trial 1 finished with

(1, 3)
(1, 41)
(1, 10)
bz_error: 3.500174261716893e-05


[I 2021-04-23 08:36:11,396] Trial 2 finished with value: 75.38100481887147 and parameters: {'Density_Feed Properties': 0.8762660474925108, 'Density_Vent Gas Production Rate and Composition': 0.5446052441985357, 'Density_Distillate (Benzene Drag) Production Rate and Composition': 0.8669908655057841, 'Density_Sidedraw (Benzene )Production Rate and Composition': 0.8836839299277046, 'Density_Bottoms Production Rate and Composition': 0.8707374716576344, 'Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr': 119.45759699836701, 'Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr': 11.683153559922127, 'Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr': 10.896309122798359, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC': 86.52808272300867, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC': 89.6343251205884, 'Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw': 979.9999872186162}. Be

(1, 2)
(1, 41)
(1, 10)
nainbz_error: 28.461385185936024
tol_error: 6.933579426054008


In [14]:
bz_error,nainbz_error,tol_error

(3.500174261716893e-05, 28.461385185936024, 6.933579426054008)

# 計算delta調幅

In [15]:
c620_op2-c620_op

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate Production Rate and Composition,Density_Sidedraw Production Rate and Composition,Density_Bottoms Production Rate and Composition,Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
140-023,-0.001353,-0.002259,0.00579,0.001288,-0.000073,-8.066153,-0.829113,-1.089537,-1.675556,0.077754


In [16]:
c660_op2-c660_op

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate (Benzene Drag) Production Rate and Composition,Density_Sidedraw (Benzene )Production Rate and Composition,Density_Bottoms Production Rate and Composition,Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
140-023,0.000625,-0.004281,-0.00619,0.000033,0.000028,89.284543,8.297503,3.585251,0.18559,0.95416


In [17]:
c670_op2-c670_op

,Density_Distillate Production Rate and Composition,Density_Bottoms Production Rate and Composition,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
140-023,-4.768372e-07,-0.00002,3.554352,0.32205,0.28437,-0.001434,0.099075


# 檢查三項約束條件

In [18]:
c620_wt2[['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]

,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
140-023,70.000035


In [19]:
na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
c660_wt2.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000

951.538614814064

In [20]:
c660_wt2['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000

3.066420573945992

In [24]:
c620_wt2.iloc[:,:41].sum(axis=1)

140-023    100.0
dtype: float64

In [25]:
c660_wt2.iloc[:,:41].sum(axis=1)

140-023    100.0
dtype: float64

In [26]:
c670_wt2.iloc[:,:41].sum(axis=1)

140-023    100.0
dtype: float64